In [1]:
from ultralytics import YOLO
import os
from IPython.display import Image, display
from IPython import display
import torch
from collections import defaultdict
import numpy as np
import cv2

In [2]:
model = YOLO('yolo11n.pt')

In [3]:
classes = ['Car', 'Cyclist', 'Pedestrian']  # Replace with your actual class names
train_path = r'D:\Chinmay\MSc - DTU\Course Material\3rd Sem\Perception\Project\YOLO\images\train'  # Updated training dataset path
valid_path = r'D:\Chinmay\MSc - DTU\Course Material\3rd Sem\Perception\Project\YOLO\images\val'  # Replace with your actual validation dataset path

yaml_file = 'names:\n'
yaml_file += '\n'.join(f'- {c}' for c in classes)
yaml_file += f'\nnc: {len(classes)}'
yaml_file += f'\ntrain: {str(train_path)}\nval: {str(valid_path)}'
with open('kitti.yaml', 'w') as f:
    f.write(yaml_file)

In [4]:
train_results = model.train(
    data='kitti.yaml', # Path to the dataset configuration file,
    epochs=300, # Increased epochs
    patience=0, # No early stopping
    batch=16,# Batch size
    imgsz=640,  # Image size
    mixup=0.1,# Mixup augmentation factor
    project='Perception-yolov11n-kitti', # Project name
    name='Test Tracking',# Naming the experiment
    device=0)# Use GPU if available, specified as a string

New https://pypi.org/project/ultralytics/8.3.31 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.29  Python-3.10.14 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
engine\trainer: task=detect, mode=train, model=yolo11n.pt, data=kitti.yaml, epochs=3, time=None, patience=0, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=Perception-yolov11n-kitti, name=Test Tracking, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, 

train: Scanning D:\Chinmay\MSc - DTU\Course Material\3rd Sem\Perception\Project\YOLO\labels\train.cache... 7342 images, 666 backgrounds, 0 corrupt: 100%|██████████| 8008/8008 [00:00<?, ?it/s]
val: Scanning D:\Chinmay\MSc - DTU\Course Material\3rd Sem\Perception\Project\YOLO\labels\val.cache... 145 images, 0 backgrounds, 0 corrupt: 100%|██████████| 145/145 [00:00<?, ?it/s]


Plotting labels to Perception-yolov11n-kitti\Test Tracking\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001429, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to Perception-yolov11n-kitti\Test Tracking
Starting training for 3 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/3      3.05G       1.51      1.572      1.136         70        640: 100%|██████████| 501/501 [03:55<00:00,  2.13it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:01<00:00,  3.54it/s]

                   all        145        883      0.401       0.33      0.374      0.191


...

        3/3      2.95G      1.232     0.9136       1.05        139        640: 100%|██████████| 501/501 [02:30<00:00,  3.33it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:00<00:00,  5.34it/s]

                   all        145        883      0.932      0.343      0.424      0.202



3 epochs completed in 0.159 hours.
Optimizer stripped from Perception-yolov11n-kitti\Test Tracking\weights\last.pt, 5.4MB
Optimizer stripped from Perception-yolov11n-kitti\Test Tracking\weights\best.pt, 5.4MB

Validating Perception-yolov11n-kitti\Test Tracking\weights\best.pt...
Ultralytics 8.3.29  Python-3.10.14 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
YOLO11n summary (fused): 238 layers, 2,582,737 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:02<00:00,  2.32it/s]


                   all        145        883      0.932      0.343      0.425      0.202
               Cyclist         93        101          1          0     0.0662     0.0152
            Pedestrian        145        782      0.864      0.687      0.783      0.389
Speed: 0.4ms preprocess, 2.0ms inference, 0.0ms loss, 4.8ms postprocess per image
Results saved to Perception-yolov11n-kitti\Test Tracking


In [ ]:
seq_number = 1
seq_dir = r"D:\Chinmay\MSc - DTU\Course Material\3rd Sem\Perception\Project\34759_final_project_rect\seq_01\image_02\data"

image_list = sorted([f for f in os.listdir(seq_dir)])

track_history = defaultdict(lambda: [])

for image in image_list:
    results = model.track(image, persist=True)
    # boxes = results[0].boxes.xywh.cpu()
    # track_ids = results[0].boxes.id.int().cpu().tolist()

FileNotFoundError: 000000.png does not exist